In [1]:
import pandas as pd
from keras.models import load_model
import numpy as np

Using TensorFlow backend.


In [2]:
# print("Enter output filename")
# output = input()
# print("Enter test data filename")
# file_name = input()

In [3]:

# df = pd.read_csv(file_name)

# # Drop variable
# df = df.drop(columns=['open', 'high', 'low', 'close', 'volume', 'dividend', 'split'])

# # Split data by IDs and write to different files
# dfs = dict(tuple(df.groupby('id')))
# list_df = [dfs[x] for x in dfs]
# for index, df in enumerate(list_df):
#     df['moving_average'] = df['adjusted_close'].rolling(5).mean()
#     df.to_csv("data_by_id/" + str(index) + ".csv", index=False)

In [4]:
import os
from sklearn.preprocessing import MinMaxScaler

directory_in_str = os.getcwd()
directory_in_str += '/data_by_id'

In [5]:
def change_category(now, week_from_now):
    ratio = float(week_from_now)/now
    percentile = (ratio - 1) * 100
    if abs(percentile) >= 5:
        return np.sign(percentile) * 3
    elif abs(percentile) >= 3:
        return np.sign(percentile) * 2
    elif abs(percentile) >= 2:
        return np.sign(percentile)
    else:
        return 0

In [6]:
def LSTM_predict(filename):
    filename = filename # 'Just_1/' + 
    print(filename)
    df = pd.read_csv(filename, index_col="time",parse_dates=True)
    df = df[4:]
    column = df['adjusted_open'].count()
    if (column > 60):
        return actual_predict(df, column)
    return None

In [7]:
def actual_predict(df, column):
        temp = df.copy()
        training_set = df.drop(['id', 'adjusted_volume'], axis=1)
        # Feature Scaling
        sc = MinMaxScaler(feature_range = (0, 1))
        training_set_scaled = sc.fit_transform(training_set)
        training_set_scaled.shape
        # Creating a data structure with 60 timesteps and 1 output
        X_test = []
        for i in range(60, column):
            X_test.append(training_set_scaled[i-60:i, :])
        X_test = np.array(X_test)
        # Reshaping
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 5))
        regressor = load_model('../model.h5')
        y = regressor.predict(X_test)
        return temp, y

In [8]:
 def change_percent(together_df):
    together_df.rename(index=str, columns={3: "price"}, inplace = True)
    together_df = together_df.assign(actual_lag_5 = lambda x: x.price.shift(-5))
    together_df = together_df.assign(actual_percent = lambda x: (x.actual_lag_5 - x.price)/x.price)
    together_df = together_df.assign(up_two = lambda x: (x.actual_percent > 0.02).astype(int))
    together_df = together_df.assign(up_three = lambda x: (x.actual_percent > 0.03).astype(int))
    together_df = together_df.assign(up_five = lambda x: (x.actual_percent > 0.05).astype(int))
    
    together_df = together_df.assign(down_two = lambda x: (x.actual_percent < 0.02).astype(int))
    together_df = together_df.assign(down_three = lambda x: (x.actual_percent < 0.03).astype(int))
    together_df = together_df.assign(down_five = lambda x: (x.actual_percent < 0.05).astype(int))
    together_df = together_df.drop([0, 1, 2, 'price', 4, 'actual_lag_5', 'actual_percent'], axis=1)
    return together_df

In [9]:
def write_to_file(ids, y):
    ids = ids.drop(['adjusted_close', 'adjusted_open', 'adjusted_high', 'adjusted_low', 'adjusted_volume', 'moving_average'], axis=1)
    time_index = ids[60:].index
    y.index = time_index
    stock_id = ids.loc[ids.index[1],'id']
    df_all_cols = pd.concat((ids[60:], y), axis = 1)
    with open("../outputs/" + str(stock_id) + ".txt", "a") as myfile:
        for i in df_all_cols.index:
            if df_all_cols.loc[i,'up_two'] > 0 and df_all_cols.loc[i,'up_three'] > 0 and df_all_cols.loc[i,'up_five'] > 0:
                new_line = str(str(stock_id) + ' ' + str(i) + ' +5 \n' )
                myfile.write(new_line)
            elif df_all_cols.loc[i,'up_two'] > 0 and df_all_cols.loc[i,'up_three'] > 0:
                new_line = str(str(stock_id) + ' ' + str(i) + ' +3 \n' )
                myfile.write(new_line)
            elif df_all_cols.loc[i,'up_two'] > 0:
                new_line = str(str(stock_id) + ' ' + str(i) + ' +2 \n' )
                myfile.write(new_line)
            elif df_all_cols.loc[i,'down_two'] > 0 and df_all_cols.loc[i,'down_three'] > 0 and df_all_cols.loc[i,'down_five'] > 0:
                new_line = str(str(stock_id) + ' ' + str(i) + ' -5 \n' )
                myfile.write(new_line)
            elif df_all_cols.loc[i,'down_two'] > 0 and df_all_cols.loc[i,'down_three'] > 0:
                new_line = str(str(stock_id) + ' ' + str(i) + ' -3 \n' )
                myfile.write(new_line)
            elif df_all_cols.loc[i,'down_two'] > 0:
                new_line = str(str(stock_id) + ' ' + str(i) + ' -2 \n' )
                myfile.write(new_line)
            else:
                new_line = str(str(stock_id) + ' ' + str(i) + ' +0 \n' )
                myfile.write(new_line)

In [10]:
os.chdir('..')
os.chdir('data_by_id')
# directory = os.fsencode(directory_in_str)
directory = os.getcwd()

for file in os.listdir(directory):
    filename = os.fsdecode(file)
#     print(filename)
    if filename.endswith(".csv"):
        temp, y = LSTM_predict(filename)
        if y is not None:
            dfnew = pd.DataFrame(y)
            new = change_percent(dfnew)
            write_to_file(temp, new)

545.csv
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
223.csv
4817.csv
4803.csv
237.csv
6972.csv
8787.csv
9499.csv
11395.csv
6782.csv
579.csv
8977.csv
3822.csv
4195.csv
2282.csv
1953.csv
1947.csv
3188.csv
2296.csv
4181.csv
6796.csv
8963.csv
8988.csv
586.csv
1012.csv
8750.csv
3605.csv
11342.csv
5274.csv
5512.csv
3163.csv
11424.csv
1774.csv
9328.csv
7311.csv
8022.csv
1760.csv
2269.csv
3177.csv
11430.csv
4618.csv
3611.csv
5260.csv
8744.csv
4156.csv
3639.csv
10060.csv
2527.csv
5248.csv
9472.csv
6741.csv
1990.csv
7339.csv
1748.csv
9314.csv
10706.csv
2241.csv
11418.csv
4630.csv
10712.csv
2255.csv
4624.csv
9300.csv
6033.csv
9466.csv
8778.csv
10074.csv
2533.csv
747.csv
8591.csv
2902.csv
2916.csv
10289.csv
753.csv
9843.csv
1589.csv
2080.csv
5937.csv
4397.csv
5089.csv
7846.csv
6580.csv
7852.csv
6594.csv
4383.csv
5923.csv
6219.csv
1576.csv
7107.csv
10538.csv

KeyboardInterrupt: 

In [ ]:
# os.chdir('..')
# os.chdir('data_by_id')
print(os.getcwd())
# directory = os.getcwd()

In [ ]:
output = '../output/' + str(id) + '.txt'

In [ ]:
print(stock_id)

In [ ]:
# stock_id = dataset.loc[dataset.index[1],'id']
print(os.getcwd())

In [ ]:
print(filename)